In [22]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import time

# Utils

In [ ]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

# Database

In [19]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5432,
    user="admin",
    password="admin",
    dbname="bigbrotr"
)

In [20]:
bigbrotr.connect()

In [21]:
query = "SELECT * FROM relay_metadata"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

In [ ]:
bigbrotr.close()

# relay_urls

In [ ]:
import pandas as pd
relays_url = pd.read_csv("../seed/relays_url.csv")
relays_url = relays_url.groupby('relay_url').agg({'count': 'sum'}).reset_index()
relays_url = relays_url.sort_values(by='count', ascending=False).reset_index(drop=True)
relays_url.to_csv("../seed/relays_url.csv", index=False)

# compute_relay_metadata

In [23]:
from relay import Relay
from aiohttp import ClientSession

relay = Relay('wss://relay.nostr.hu')

async def fetch_nip11_metadata(relay_id, session, timeout):
    headers = {'Accept': 'application/nostr+json'}
    for schema in ['https://', 'http://']:
        try:
            async with session.get(schema + relay_id, headers=headers, timeout=timeout) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    pass
        except Exception as e:
            pass
    return None

relay_id = relay.url.removeprefix('wss://')
nip11_raw = None
async with ClientSession() as session:
    nip11_raw = await fetch_nip11_metadata(relay_id, session, 10)
nip11_raw

{'name': 'nostr.hu relay',
 'description': 'Only notes in utxo WoT',
 'pubkey': 'cfd7df62799a22e384a4ab5da8c4026c875b119d0f47c2716b20cdac9cc1f1a6',
 'contact': 'https://nostr.hu',
 'supported_nips': [1, 11, 42, 70, 86, 9],
 'software': 'https://github.com/bitvora/wot-relay',
 'version': 'v0.1.16',
 'icon': 'https://nostr.hu/mano64.jpg'}

In [31]:
from compute_relay_metadata import compute_relay_metadata

sec, pub = utils.generate_nostr_keypair()
metadata = await compute_relay_metadata(relay, sec, pub, socks5_proxy_url=None, timeout=10)
metadata.to_dict()

Error: 429, message='Invalid response status', url='wss://relay.nostr.hu'


{'relay': Relay(url=wss://relay.nostr.hu, network=clearnet),
 'generated_at': 1747542684,
 'connection_success': False,
 'nip11_success': True,
 'readable': None,
 'writable': None,
 'rtt': None,
 'name': 'nostr.hu relay',
 'description': 'Only notes in utxo WoT',
 'banner': None,
 'icon': 'https://nostr.hu/mano64.jpg',
 'pubkey': 'cfd7df62799a22e384a4ab5da8c4026c875b119d0f47c2716b20cdac9cc1f1a6',
 'contact': 'https://nostr.hu',
 'supported_nips': None,
 'software': 'https://github.com/bitvora/wot-relay',
 'version': 'v0.1.16',
 'privacy_policy': None,
 'terms_of_service': None,
 'limitation': None,
 'extra_fields': None}